In [17]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
import requests as req
import re
import seaborn as sn
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
import csv
pd.set_option("display.max_colwidth",None)

# Testing site

In [18]:
nb_page = 1
year = 2020
url = f'https://www.cvedetails.com/vulnerability-list.php?vendor_id=0&product_id=0&version_id=0&page={nb_page}&hasexp=0&opdos=0&opec=0&opov=0&opcsrf=0&opgpriv=0&opsqli=0&opxss=0&opdirt=0&opmemc=0&ophttprs=0&opbyp=0&opfileinc=0&opginf=0&cvssscoremin=0&cvssscoremax=0&year={year}&month=0&cweid=0&order=1&trc=18325&sha=e119aac167607cc9cbfbc74a520f6c4fb434a151'
html = req.get(url)
soup = bs(html.content,'lxml')
soup.find('table',class_="searchresults sortable").th
soup.find('table',class_="searchresults sortable").td

<td class="num">
<a name="y2020"> </a>
											1					</td>

In [19]:
table = soup.find('table',class_="searchresults sortable")
tr0 = table.tr
cols = tr0.findAll('th')

for c in cols:
    print(c.text.strip())

#
CVE ID
CWE ID
# of Exploits
Vulnerability Type(s)
Publish Date
Update Date
Score
Gained Access Level
Access
Complexity
Authentication
Conf.
Integ.
Avail.


In [20]:
# Extract the range number of pages
pagingb = soup.find('div',id="pagingb")
pages_nums = pagingb.findAll('a')
pn_list = []
for p in pages_nums:
    pn_list.append(p.text)
print('total pages available in this year is : ' ,pn_list[-1])
print(range(len(pn_list)))
int(pn_list[-1])

total pages available in this year is :  367
range(0, 367)


367

In [21]:
# Extract cve summary 
table = soup.find('table', attrs={'id': 'vulnslisttable'})
rows = table.find_all('tr')
s=[]
for i,row in enumerate(rows):
    col = row.find('td',class_="cvesummarylong")
    if col is not None:
        s.append(col.text.strip())
        
pd.set_option("display.max_colwidth",None)
pd.DataFrame(s)

,0
0,"An issue was discovered in Foxit Reader before 10.1.1 (and before 4.1.1 on macOS) and PhantomPDF before 9.7.5 and 10.x before 10.1.1 (and before 4.1.1 on macOS). An attacker can spoof a certified PDF document via an Evil Annotation Attack because the products fail to consider a null value for a Subtype entry of the Annotation dictionary, in an incremental update."
1,"Seo Panel 4.8.0 allows stored XSS by an Authenticated User via the url parameter, as demonstrated by the seo/seopanel/websites.php URI."
2,"An issue was discovered in the concread crate before 0.2.6 for Rust. Attackers can cause an ARCache<K,V> data race by sending types that do not implement Send/Sync."
3,An issue was discovered in the thex crate through 2020-12-08 for Rust. Thex<T> allows cross-thread data races of non-Send types.
4,An issue was discovered in the nanorand crate before 0.5.1 for Rust. It caused any random number generator (even ChaCha) to return all zeroes because integer truncation was mishandled.
5,An issue was discovered in the magnetic crate before 2.0.1 for Rust. MPMCConsumer and MPMCProducer allow cross-thread sending of a non-Send type.
6,An issue was discovered in the try-mutex crate before 0.3.0 for Rust. TryMutex<T> allows cross-thread sending of a non-Send type.
7,An issue was discovered in the ordered-float crate before 1.1.1 and 2.x before 2.0.1 for Rust. A NotNan value can contain a NaN.
8,An issue was discovered in the mio crate before 0.7.6 for Rust. It has false expectations about the std::net::SocketAddr memory representation.
9,An issue was discovered in the miow crate before 0.3.6 for Rust. It has false expectations about the std::net::SocketAddr memory representation.


In [22]:
range(1,int(pn_list[-1])+1)

range(1, 368)

#  Scraping functions :

#  1- CVE Summary Scraper

In [58]:
def cve_summary_scraper(year=2020):
    # Extract the number of pages available in the entered year
    #url_for_pages = f'year={year}&month=0&cweid=0&order=1&trc=20171&sha=5865744384614acda35d0681e9e72ad105514ab2'
    url_for_pages = f'https://www.cvedetails.com/vulnerability-list.php?vendor_id=0&product_id=0&version_id=0&page=2&hasexp=0&opdos=0&opec=0&opov=0&opcsrf=0&opgpriv=0&opsqli=0&opxss=0&opdirt=0&opmemc=0&ophttprs=0&opbyp=0&opfileinc=0&opginf=0&cvssscoremin=0&cvssscoremax=0&year={year}&month=0&cweid=0&order=1&trc=17344&sha=06eae532a6d5b0054c1986953e99779c5827e3a7'
    html_for_pages = req.get(url_for_pages)
    soup_for_pages = bs(html_for_pages.content,'lxml')
    pagingb = soup_for_pages.find('div',id="pagingb")
    pages_nums = pagingb.findAll('a')
    pn_list = []
    for p in pages_nums:
        pn_list.append(p.text)
    nb_pages = pn_list[-1]

    # ...
    s=[]
    for page in range(1,int(nb_pages)+1):
        #url = f'year={year}&month=0&cweid=0&order=1&trc=18325&sha=e119aac167607cc9cbfbc74a520f6c4fb434a151'
        url = f'https://www.cvedetails.com/vulnerability-list.php?vendor_id=0&product_id=0&version_id=0&page={page}\
        &hasexp=0&opdos=0&opec=0&opov=0&opcsrf=0&opgpriv=0&opsqli=0&opxss=0&opdirt=0&opmemc=0&ophttprs=0&opbyp=0&opfileinc=\
        0&opginf=0&cvssscoremin=0&cvssscoremax=0&year={year}&month=0&cweid=0&order=1&trc=17344&sha=06eae532a6d5b0054c1986953e99779c5827e3a7'
        html = req.get(url)
        soup = bs(html.content,'lxml')

        table = soup.find('table', attrs={'id': 'vulnslisttable'})
        rows = table.find_all('tr')
        for row in rows:
            col = row.find('td',class_="cvesummarylong")
            if col is not None:
                s.append(col.text.strip())

    return pd.DataFrame(s,columns=['CVE Summary'])


##### 2020

In [24]:
# cve_summary_2020 = cve_summary_scraper(year=2020) # 18325 rows
# cve_summary_2020.to_csv('cve_summary_2020.csv')
cve_summary_2020 = pd.read_csv('cve_summary_2020.csv')
cve_summary_2020

,Unnamed: 0,CVE Summary
0,0,"An issue was discovered in Foxit Reader before 10.1.1 (and before 4.1.1 on macOS) and PhantomPDF before 9.7.5 and 10.x before 10.1.1 (and before 4.1.1 on macOS). An attacker can spoof a certified PDF document via an Evil Annotation Attack because the products fail to consider a null value for a Subtype entry of the Annotation dictionary, in an incremental update."
1,1,"Seo Panel 4.8.0 allows stored XSS by an Authenticated User via the url parameter, as demonstrated by the seo/seopanel/websites.php URI."
2,2,"An issue was discovered in the concread crate before 0.2.6 for Rust. Attackers can cause an ARCache<K,V> data race by sending types that do not implement Send/Sync."
3,3,An issue was discovered in the thex crate through 2020-12-08 for Rust. Thex<T> allows cross-thread data races of non-Send types.
4,4,An issue was discovered in the nanorand crate before 0.5.1 for Rust. It caused any random number generator (even ChaCha) to return all zeroes because integer truncation was mishandled.
...,...,...
18320,18320,"The Linux kernel before 2.4.36-rc1 has a race condition. It was possible to bypass systrace policies by flooding the ptraced process with SIGCONT signals, which can can wake up a PTRACED process."
18321,18321,Systrace before 1.6.0 has insufficient escape policy enforcement.
18322,18322,NetworkManager 0.9.x does not pin a certificate's subject to an ESSID when 802.11X authentication is used.
18323,18323,Simple Machine Forum (SMF) versions 1.0.4 and earlier have an SQL injection vulnerability that allows remote attackers to inject arbitrary SQL statements.


##### 2021

In [41]:
# cve_summary_2021 = cve_summary_scraper(year=2021) # 20171 rows
# cve_summary_2021.to_csv('cve_summary_2021.csv')
cve_summary_2021 = pd.read_csv('cve_summary_2021.csv')
cve_summary_2021

,Unnamed: 0,CVE Summary
0,0,An issue was discovered in gif2apng 1.9. There is a heap-based buffer overflow in the main function. It allows an attacker to write 2 bytes outside the boundaries of the buffer.
1,1,"An issue was discovered in gif2apng 1.9. There is a heap-based buffer overflow within the main function. It allows an attacker to write data outside of the allocated buffer. The attacker has control over a part of the address that data is written to, control over the written data, and (to some extent) control over the amount of data that is written."
2,2,An issue was discovered in gif2apng 1.9. There is a heap-based buffer overflow vulnerability in the DecodeLZW function. It allows an attacker to write a large amount of arbitrary data outside the boundaries of a buffer.
3,3,"An issue was discovered in gif2apng 1.9. There is a stack-based buffer overflow involving a while loop. An attacker has little influence over the data written to the stack, making it unlikely that the flow of control can be subverted."
4,4,"An issue was discovered in gif2apng 1.9. There is a stack-based buffer overflow involving a for loop. An attacker has little influence over the data written to the stack, making it unlikely that the flow of control can be subverted."
...,...,...
20166,20166,"It was found that various OpenID Providers (OPs) had TLS Server Certificates that used weak keys, as a result of the Debian Predictable Random Number Generator (CVE-2008-0166). In combination with the DNS Cache Poisoning issue (CVE-2008-1447) and the fact that almost all SSL/TLS implementations do not consult CRLs (currently an untracked issue), this means that it is impossible to rely on these OPs."
20167,20167,"Mounting /proc filesystem via chroot command silently mounts it in read-write mode. The user could bypass the chroot environment and gain write access to files, he would never have otherwise."
20168,20168,A flaw in Mozilla's embedded certificate code might allow web sites to install root certificates on devices without user approval.
20169,20169,"The Diffie-Hellman Key Agreement Protocol allows remote attackers (from the client side) to send arbitrary numbers that are actually not public keys, and trigger expensive server-side DHE modular-exponentiation calculations, aka a D(HE)ater attack. The client needs very little CPU resources and network bandwidth. The attack may be more disruptive in cases where a client can require a server to select its largest supported key size. The basic attack scenario is that the client must claim that it can only communicate with DHE, and the server must be configured to allow DHE."


##### 2022

In [43]:
# cve_summary_2022 = cve_summary_scraper(year=2022) # 25227 rows
# cve_summary_2022.to_csv('cve_summary_2022.csv')
cve_summary_2022 = pd.read_csv('cve_summary_2022.csv')
cve_summary_2022

,Unnamed: 0,CVE Summary
0,0,"Certain NETGEAR devices are affected by a buffer overflow by an unauthenticated attacker. This affects RAX40 before 1.0.2.60, RAX35 before 1.0.2.60, R6400v2 before 1.0.4.122, R6700v3 before 1.0.4.122, R6900P before 1.3.3.152, R7000P before 1.3.3.152, R7000 before 1.0.11.136, R7960P before 1.4.4.94, and R8000P before 1.4.4.94."
1,1,"An issue was discovered in Mellium mellium.im/sasl before 0.3.1. When performing SCRAM-based SASL authentication, if the remote end advertises support for channel binding, no random nonce is generated (instead, the nonce is empty). This causes authentication to fail in the best case, but (if paired with a remote end that does not validate the length of the nonce) could lead to insufficient randomness being used during authentication."
2,2,TP-Link TL-WR902AC devices through V3 0.9.1 allow remote authenticated attackers to execute arbitrary code or cause a Denial of Service (DoS) by uploading a crafted firmware update because the signature check is inadequate.
3,3,"Heimdall Application Dashboard through 2.5.4 allows reflected and stored XSS via ""Application name"" to the ""Add application"" page. The stored XSS will be triggered in the ""Application list"" page."
4,4,"The Nintendo NetworkBuffer class, as used in Animal Crossing: New Horizons before 2.0.6 and other products, allows remote attackers to execute arbitrary code via a large UDP packet that causes a buffer overflow, aka ENLBufferPwn. The victim must join a game session with the attacker. Other affected products include Mario Kart 7 before 1.2, Mario Kart 8, Mario Kart 8 Deluxe before 2.1.0, ARMS before 5.4.1, Splatoon, Splatoon 2 before 5.5.1, Splatoon 3 before late 2022, Super Mario Maker 2 before 3.0.2, and Nintendo Switch Sports before late 2022."
...,...,...
25222,25222,"A flaw was found in StarWind iSCSI target. An attacker could script standard iSCSI Initiator operation(s) to exhaust the StarWind service socket, which could lead to denial of service. This affects iSCSI SAN (Windows Native) Version 3.2.2 build 2007-02-20."
25223,25223,** UNSUPPORTED WHEN ASSIGNED ** A vulnerability was found in Netegrity SiteMinder up to 4.5.1 and classified as critical. Affected by this issue is the file /siteminderagent/pwcgi/smpwservicescgi.exe of the component Login. The manipulation of the argument target leads to an open redirect. The exploit has been disclosed to the public and may be used. NOTE: This vulnerability only affects products that are no longer supported by the maintainer.
25224,25224,** UNSUPPORTED WHEN ASSIGNED ** A vulnerability was found in ISS BlackICE PC Protection. It has been rated as problematic. Affected by this issue is the Update Handler. The manipulation with an unknown input leads to cross site scripting. The attack may be launched remotely. The exploit has been disclosed to the public and may be used. NOTE: This vulnerability only affects products that are no longer supported by the maintainer.
25225,25225,** UNSUPPORTED WHEN ASSIGNED ** A vulnerability was found in ISS BlackICE PC Protection. It has been declared as problematic. Affected by this vulnerability is the component Update Handler which allows cleartext transmission of data. NOTE: This vulnerability only affects products that are no longer supported by the maintainer.


#### 2023

In [47]:
# cve_summary_2023 = cve_summary_scraper(year=2023) # 10541 rows (until May)
# cve_summary_2023.to_csv('cve_summary_2023.csv')
cve_summary_2023 = pd.read_csv('cve_summary_2023.csv')
cve_summary_2023

,Unnamed: 0,CVE Summary
0,0,LuaTeX before 1.17.0 enables the socket library by default.
1,1,"In Qt before 5.15.14, 6.0.x through 6.2.x before 6.2.9, and 6.3.x through 6.5.x before 6.5.1, QtSvg QSvgFont m_unitsPerEm initialization is mishandled."
2,2,"VideoLAN dav1d before 1.2.0 has a thread_task.c race condition that can lead to an application crash, related to dav1d_decode_frame_exit."
3,3,"An issue was discovered in Veritas InfoScale Operations Manager (VIOM) before 7.4.2.800 and 8.x before 8.0.410. The InfoScale VIOM web application is vulnerable to SQL Injection in some of the areas of the application. This allows attackers to submit arbitrary SQL commands on the back-end database to create, read, update, or delete any sensitive data stored in the database."
4,4,"An issue was discovered in Veritas InfoScale Operations Manager (VIOM) before 7.4.2.800 and 8.x before 8.0.410. The VIOM web application does not validate user-supplied data and appends it to OS commands and internal binaries used by the application. An attacker with root/administrator level privileges can leverage this to read sensitive data stored on the servers, modify data or server configuration, and delete data or application configuration."
...,...,...
10536,10536,A vulnerability has been found in cfire24 ajaxlife up to 0.3.2 and classified as problematic. This vulnerability affects unknown code. The manipulation leads to cross site scripting. The attack can be initiated remotely. Upgrading to version 0.3.3 is able to address this issue. The name of the patch is 9fb53b67312fe3f4336e01c1e3e1bedb4be0c1c8. It is recommended to upgrade the affected component. VDB-222286 is the identifier assigned to this vulnerability.
10537,10537,"A vulnerability, which was classified as critical, has been found in web-cyradm. Affected by this issue is some unknown functionality of the file auth.inc.php. The manipulation of the argument login/login_password/LANG leads to sql injection. The attack may be launched remotely. The name of the patch is 2bcbead3bdb5f118bf2c38c541eaa73c29dcc90f. It is recommended to apply a patch to fix this issue. The identifier of this vulnerability is VDB-217640."
10538,10538,A vulnerability classified as problematic has been found in web-cyradm. This affects an unknown part of the file search.php. The manipulation of the argument searchstring leads to sql injection. It is recommended to apply a patch to fix this issue. The identifier VDB-217449 was assigned to this vulnerability.
10539,10539,"A carefully crafted If: request header can cause a memory read, or write of a single zero byte, in a pool (heap) memory location beyond the header value sent. This could cause the process to crash. This issue affects Apache HTTP Server 2.4.54 and earlier."


#### 2019

In [56]:
# cve_summary_2019 = cve_summary_scraper(year=2019) # 17344 rows
# cve_summary_2019.to_csv('cve_summary_2019.csv')
cve_summary_2019 = pd.read_csv('cve_summary_2019.csv')
cve_summary_2019

,Unnamed: 0,CVE Summary
0,0,invenio-previewer before 1.0.0a12 allows XSS.
1,1,Discourse before 2.3.0 and 2.4.x before 2.4.0.beta3 lacks a confirmation screen when logging in via an email link.
2,2,Discourse before 2.3.0 and 2.4.x before 2.4.0.beta3 lacks a confirmation screen when logging in via a user-api OTP.
3,3,ASH-AIO before 2.0.0.3 allows an open redirect.
4,4,graphql-engine (aka Hasura GraphQL Engine) before 1.0.0-beta.3 mishandles the audience check while verifying JWT.
...,...,...
17339,17339,Cross-site scripting (XSS) vulnerability in websieve v0.62 allows remote attackers to inject arbitrary web script or HTML code in the web user interface.
17340,17340,Zoo 2.10 has Directory traversal
17341,17341,go.cgi in GoScript 2.0 allows remote attackers to execute arbitrary commands via shell metacharacters in the (1) query string or (2) artarchive parameter.
17342,17342,Snoopy before 2.0.0 has a security hole in exec cURL


# 2- CVE Info Scraper

In [59]:
def cve_info_scraper(filename,year=2020):
    with open(filename+'.csv', mode='w', newline='') as file:

        # Define URL # Extract total number of pages for looping them
        #url_for_pages = f'year={year}&month=0&cweid=0&order=1&trc=10541&sha=4b2dc34ae8768f1ab3e386ccc8fea84500014957'
        url_for_pages = f'https://www.cvedetails.com/vulnerability-list.php?vendor_id=0&product_id=0&version_id=0&page=2&hasexp=0&opdos=0&opec=0&opov=0&opcsrf=0&opgpriv=0&opsqli=0&opxss=0&opdirt=0&opmemc=0&ophttprs=0&opbyp=0&opfileinc=0&opginf=0&cvssscoremin=0&cvssscoremax=0&year={year}&month=0&cweid=0&order=1&trc=17344&sha=06eae532a6d5b0054c1986953e99779c5827e3a7'
        html_for_pages = req.get(url_for_pages)
        soup_for_pages = bs(html_for_pages.content,'lxml')
        pagingb = soup_for_pages.find('div',id="pagingb")
        pages_nums = pagingb.findAll('a')
        pn_list = []
        for p in pages_nums:
            pn_list.append(p.text)
        nb_page_total = pn_list[-1]
        
        # Extract and Store in CSV
        writer = csv.writer(file)
        writer.writerow(['CVE ID', 'CWE ID', '# of Exploits', 'Vulnerability Type(s)',
                         'Publish Date', 'Update Date', 'Score', 'Gained Access Level',
                         'Access', 'Complexity', 'Authentication', 'Conf.', 'Integ.', 'Avail'])
        
        # All pages 
        for page in range(1,int(nb_page_total)+1):
            #url = f'year={year}&month=0&cweid=0&order=1&trc=25227&sha=45d566efbc1f55ce107b057217e11d794a7bc4fb'
            url = f'https://www.cvedetails.com/vulnerability-list.php?vendor_id=0&product_id=0&version_id=0&page={page}&hasexp=0&opdos=0&opec=0&opov=0&opcsrf=0&opgpriv=0&opsqli=0&opxss=0&opdirt=0&opmemc=0&ophttprs=0&opbyp=0&opfileinc=0&opginf=0&cvssscoremin=0&cvssscoremax=0&year={year}&month=0&cweid=0&order=1&trc=17344&sha=06eae532a6d5b0054c1986953e99779c5827e3a7'
            html = req.get(url)
            soup = bs(html.content,'lxml')
            table = soup.find('table', attrs={'id': 'vulnslisttable'})
            rows = table.find_all('tr')
            
            # loop through the table rows and write each row to the CSV file
            for row in rows:
            
                cols = row.find_all('td')
                if len(cols) < 14:  # Check if the number of columns is less than expected
                    continue  # Skip this row
                cve_id = cols[1].text.strip()
                cwe_id = cols[2].text.strip()
                num_exploits = cols[3].text.strip()
                vuln_types = cols[4].text.strip()
                pub_date = cols[5].text.strip() 
                update_date = cols[6].text.strip()
                score = cols[7].text.strip()
                gain_access = cols[8].text.strip()
                access = cols[9].text.strip()
                complexity = cols[10].text.strip()
                auth = cols[11].text.strip()
                conf = cols[12].text.strip()
                integ = cols[13].text.strip()
                avail = cols[14].text.strip()

                # write the row to the CSV file
                writer.writerow([cve_id, cwe_id, num_exploits, vuln_types, pub_date, 
                                 update_date, score, gain_access, access, complexity, 
                                 auth, conf, integ, avail])
                
    return pd.read_csv(filename+'.csv')


##### 2020

In [46]:
# cve_info = cve_info_scraper('cve_info_2020',year=2020) #18325 row
cve_info_2020 = pd.read_csv('cve_info_2020.csv')
cve_info_2020

,CVE ID,CWE ID,# of Exploits,Vulnerability Type(s),Publish Date,Update Date,Score,Gained Access Level,Access,Complexity,Authentication,Conf.,Integ.,Avail
0,CVE-2020-35931,754.0,NaN,NaN,2020-12-31,2021-09-08,6.8,None,Remote,Medium,Not required,Partial,Partial,Partial
1,CVE-2020-35930,79.0,NaN,XSS,2020-12-31,2021-01-05,3.5,None,Remote,Medium,???,None,Partial,None
2,CVE-2020-35928,362.0,NaN,NaN,2020-12-31,2021-01-06,1.9,None,Local,Medium,Not required,None,None,Partial
3,CVE-2020-35927,NaN,NaN,NaN,2020-12-31,2021-01-06,2.1,None,Local,Low,Not required,None,None,Partial
4,CVE-2020-35926,338.0,NaN,NaN,2020-12-31,2021-07-21,7.5,None,Remote,Low,Not required,Partial,Partial,Partial
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18320,CVE-2007-4774,362.0,NaN,Bypass,2020-01-15,2020-02-04,4.3,None,Remote,Medium,Not required,None,Partial,None
18321,CVE-2007-4773,415.0,NaN,NaN,2020-01-15,2020-01-23,7.5,None,Remote,Low,Not required,Partial,Partial,Partial
18322,CVE-2006-7246,295.0,NaN,NaN,2020-01-27,2020-01-31,3.2,None,Local Network,High,Not required,Partial,Partial,None
18323,CVE-2005-4891,89.0,NaN,Sql,2020-01-15,2020-01-21,7.5,None,Remote,Low,Not required,Partial,Partial,Partial


#### 2021

In [33]:
# cve_info_2021 = cve_info_scraper('cve_info_2021',year=2021) #20171 row  
cve_info_2021 = pd.read_csv('cve_info_2021.csv')
cve_info_2021

,CVE ID,CWE ID,# of Exploits,Vulnerability Type(s),Publish Date,Update Date,Score,Gained Access Level,Access,Complexity,Authentication,Conf.,Integ.,Avail
0,CVE-2021-45911,787.0,NaN,Overflow,2021-12-28,2022-03-24,6.8,None,Remote,Medium,Not required,Partial,Partial,Partial
1,CVE-2021-45910,787.0,NaN,Overflow,2021-12-28,2022-03-24,6.8,None,Remote,Medium,Not required,Partial,Partial,Partial
2,CVE-2021-45909,787.0,NaN,Overflow,2021-12-28,2022-04-06,6.8,None,Remote,Medium,Not required,Partial,Partial,Partial
3,CVE-2021-45908,787.0,NaN,Overflow,2021-12-28,2022-01-06,6.8,None,Remote,Medium,Not required,Partial,Partial,Partial
4,CVE-2021-45907,787.0,NaN,Overflow,2021-12-28,2022-01-06,6.8,None,Remote,Medium,Not required,Partial,Partial,Partial
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20166,CVE-2008-3280,338.0,NaN,NaN,2021-05-21,2021-05-27,4.3,None,Remote,Medium,Not required,Partial,None,None
20167,CVE-2008-2544,668.0,NaN,Bypass,2021-05-27,2021-06-07,2.1,None,Local,Low,Not required,None,Partial,None
20168,CVE-2007-5967,295.0,NaN,NaN,2021-05-17,2021-05-26,4.3,None,Remote,Medium,Not required,None,Partial,None
20169,CVE-2002-20001,400.0,NaN,NaN,2021-11-11,2022-11-09,5.0,None,Remote,Low,Not required,None,None,Partial


#### 2022

In [51]:
# cve_info_2022 = cve_info_scraper('cve_info_2022',year=2022) #25227 row
cve_info_2022 = pd.read_csv('cve_info_2022.csv')
cve_info_2022

,CVE ID,CWE ID,# of Exploits,Vulnerability Type(s),Publish Date,Update Date,Score,Gained Access Level,Access,Complexity,Authentication,Conf.,Integ.,Avail
0,CVE-2022-48196,120.0,NaN,Overflow,2022-12-30,2023-01-10,0.0,None,???,???,???,???,???,???
1,CVE-2022-48195,287.0,NaN,NaN,2022-12-31,2023-01-09,0.0,None,???,???,???,???,???,???
2,CVE-2022-48194,434.0,NaN,DoS Exec Code,2022-12-30,2023-04-03,0.0,None,???,???,???,???,???,???
3,CVE-2022-47968,79.0,NaN,XSS,2022-12-27,2023-01-05,0.0,None,???,???,???,???,???,???
4,CVE-2022-47949,120.0,NaN,Exec Code Overflow,2022-12-24,2023-01-06,0.0,None,???,???,???,???,???,???
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25222,CVE-2007-20001,400.0,NaN,DoS,2022-02-06,2022-09-01,5.0,None,Remote,Low,Not required,None,None,Partial
25223,CVE-2005-10001,601.0,NaN,NaN,2022-03-28,2022-04-08,5.8,None,Remote,Medium,Not required,Partial,Partial,None
25224,CVE-2003-5003,79.0,NaN,XSS,2022-03-28,2022-04-06,4.3,None,Remote,Medium,Not required,None,Partial,None
25225,CVE-2003-5002,319.0,NaN,NaN,2022-03-28,2022-04-06,5.0,None,Remote,Low,Not required,Partial,None,None


#### 2023

In [53]:
# cve_info_2023 = cve_info_scraper('cve_info_2023',year=2023) #10541 row (until May)
cve_info_2023 = pd.read_csv('cve_info_2023.csv')
cve_info_2023

,CVE ID,CWE ID,# of Exploits,Vulnerability Type(s),Publish Date,Update Date,Score,Gained Access Level,Access,Complexity,Authentication,Conf.,Integ.,Avail
0,CVE-2023-32668,NaN,NaN,NaN,2023-05-11,2023-05-11,0.0,None,???,???,???,???,???,???
1,CVE-2023-32573,NaN,NaN,NaN,2023-05-10,2023-05-10,0.0,None,???,???,???,???,???,???
2,CVE-2023-32570,NaN,NaN,NaN,2023-05-10,2023-05-10,0.0,None,???,???,???,???,???,???
3,CVE-2023-32569,NaN,NaN,Sql,2023-05-10,2023-05-10,0.0,None,???,???,???,???,???,???
4,CVE-2023-32568,NaN,NaN,NaN,2023-05-10,2023-05-10,0.0,None,???,???,???,???,???,???
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10536,CVE-2008-10002,79.0,NaN,XSS,2023-03-05,2023-03-09,0.0,None,???,???,???,???,???,???
10537,CVE-2007-10002,89.0,NaN,Sql,2023-01-08,2023-01-12,0.0,None,???,???,???,???,???,???
10538,CVE-2007-10001,89.0,NaN,Sql,2023-01-05,2023-01-11,0.0,None,???,???,???,???,???,???
10539,CVE-2006-20001,787.0,NaN,NaN,2023-01-17,2023-01-25,0.0,None,???,???,???,???,???,???


#### 2019

In [60]:
# cve_info_2019 = cve_info_scraper('cve_info_2019',year=2019) #17344 row
cve_info_2019 = pd.read_csv('cve_info_2019.csv')
cve_info_2019

,CVE ID,CWE ID,# of Exploits,Vulnerability Type(s),Publish Date,Update Date,Score,Gained Access Level,Access,Complexity,Authentication,Conf.,Integ.,Avail
0,CVE-2019-1020019,79.0,NaN,XSS,2019-07-29,2019-07-31,4.3,None,Remote,Medium,Not required,None,Partial,None
1,CVE-2019-1020018,287.0,NaN,NaN,2019-07-29,2022-04-18,7.5,None,Remote,Low,Not required,Partial,Partial,Partial
2,CVE-2019-1020017,NaN,NaN,NaN,2019-07-29,2023-03-03,5.0,None,Remote,Low,Not required,None,Partial,None
3,CVE-2019-1020016,601.0,NaN,NaN,2019-07-29,2019-08-01,5.8,None,Remote,Medium,Not required,Partial,Partial,None
4,CVE-2019-1020015,20.0,NaN,NaN,2019-07-29,2021-07-21,5.0,None,Remote,Low,Not required,None,Partial,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17339,CVE-2005-2350,79.0,NaN,XSS,2019-11-01,2019-11-04,4.3,None,Remote,Medium,Not required,None,Partial,None
17340,CVE-2005-2349,22.0,NaN,Dir. Trav.,2019-10-28,2019-11-01,5.0,None,Remote,Low,Not required,Partial,None,None
17341,CVE-2004-2776,NaN,NaN,Exec Code,2019-12-31,2020-01-14,7.5,None,Remote,Low,Not required,Partial,Partial,Partial
17342,CVE-2002-2444,20.0,NaN,NaN,2019-10-28,2019-11-01,7.5,None,Remote,Low,Not required,Partial,Partial,Partial


In [1]:

def alldata(year):
    df1 = cve_info_scraper(str(year)+'_data',year)
    df2 = cve_summary_scraper(year)
    data = pd.concat([df1,df2],axis=1)
    return data.to_csv(str(year)+'_data.csv')



CWE ID = identifient de chaque vulnerability (we will use it to solve NaN value exist in Vulnerability Type(s) column)

data : https://www.cvedetails.com/cwe-definitions.php